In [97]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time, os
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline

num_epochs = 20

log_interval = 1000

In [72]:
# GPU setup
use_cuda = torch.cuda.is_available()
print("Using {}".format('GPU' if use_cuda else 'CPU'))

Using GPU


In [73]:
from torch.utils.data import Dataset

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def read_labels(label_file):
    labels = np.zeros(50000).astype(np.uint8)
    with open(label_file, 'r') as f:
        n, header_seen = 0, False
        for line in f:
            if not header_seen:
                header_seen = True
                continue
            labels[n] = int(line.strip().split(',')[1])
            n += 1
    return labels

train_labels = read_labels("dataset/train_labels.csv")

train_images = np.load("dataset/train_data.npy")

num_train = len(train_images)
indices = list(range(num_train))
split = 10000

# shuffle data
np.random.seed(6825)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

transform = torchvision.transforms.Compose(
    [torchvision.transforms.Normalize((0.5,), (0.5,))])

class CIFAR10Dataset(Dataset):
    def __init__(self, lbls, imgs, transform=None):
        self.transform=transform
        self.samples = [(torch.from_numpy(imgs[idx])*2.0/255.0 - 1,lbls[idx].item()) for idx in range(len(train_val_labels))]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


trainset = CIFAR10Dataset(train_labels,train_images,transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=50, sampler=train_sampler, shuffle=False)

validloader = torch.utils.data.DataLoader(trainset, batch_size=50, sampler=valid_sampler, shuffle=False)

In [114]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.conv1_bn = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 32, 5)
        self.conv2_bn = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv3_bn = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 8 * 8, 120)
        self.fc1_bn = nn.BatchNorm1d(120)        
        self.dropout1 = nn.Dropout(p=0.5)        
        self.fc2 = nn.Linear(120, 84)
        self.fc2_bn = nn.BatchNorm1d(84)
        self.dropout2 = nn.Dropout(p=0.5)        
        self.fc3 = nn.Linear(84, 84)
        self.fc3_bn = nn.BatchNorm1d(84)
        self.dropout3 = nn.Dropout(p=0.5)        
        self.fc4 = nn.Linear(84, 10)
        self.fc4_bn = nn.BatchNorm1d(10)
        self.dropout4 = nn.Dropout(p=0.5)        

        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8)
        #x = self.dropout1(x)
        x = F.relu((self.fc1(x)))
        x = F.relu((self.fc2(x)))
        x = (self.fc3(x))
        x = ((self.fc4(x)))
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.Adam([var1, var2], lr=0.0001)

In [115]:
training_acc_vect = np.zeros(num_epochs)
valid_acc_vect = np.zeros(num_epochs)

start_time = time.time()

# initialize weights and biases
nn.init.kaiming_uniform_(net.conv1.weight, nonlinearity = 'relu')
stdv = 1./np.sqrt(net.conv1.bias.size(0))
nn.init.uniform_(net.conv1.bias, -stdv, stdv)
nn.init.kaiming_uniform_(net.conv2.weight, nonlinearity = 'relu')
stdv = 1./np.sqrt(net.conv2.bias.size(0))
nn.init.uniform_(net.conv2.bias, -stdv, stdv)
nn.init.kaiming_uniform_(net.conv3.weight, nonlinearity = 'relu')
stdv = 1./np.sqrt(net.conv3.bias.size(0))
nn.init.uniform_(net.conv3.bias, -stdv, stdv)
nn.init.kaiming_uniform_(net.fc1.weight, nonlinearity = 'relu')
stdv = 1./np.sqrt(net.fc1.bias.size(0))
nn.init.uniform_(net.fc1.bias, -stdv, stdv)
nn.init.kaiming_uniform_(net.fc2.weight, nonlinearity = 'relu')
stdv = 1./np.sqrt(net.fc2.bias.size(0))
nn.init.uniform_(net.fc2.bias, -stdv, stdv)
nn.init.kaiming_uniform_(net.fc3.weight, nonlinearity = 'relu')
stdv = 1./np.sqrt(net.fc3.bias.size(0))
nn.init.uniform_(net.fc3.bias, -stdv, stdv)
nn.init.kaiming_uniform_(net.fc4.weight, nonlinearity = 'relu')
stdv = 1./np.sqrt(net.fc4.bias.size(0))
nn.init.uniform_(net.fc4.bias, -stdv, stdv)

training_error_log = np.zeros(int(num_epochs*len(train_labels)/log_interval))
training_loss_log = np.zeros(int(num_epochs*len(train_labels)/log_interval))
valid_error_vect = np.zeros(num_epochs)

if use_cuda:
    net.cuda()
    criterion.cuda()

optimizer = optim.Adam(net.parameters() , lr=0.0001) 
    
# train network
for epoch in range(num_epochs):  # loop over the dataset multiple times while training
    correct_train = 0
    total_train = 0
    correct_valid = 0    
    total_valid = 0
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data # list of [inputs, labels]
        
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()  
        
        #print(labels)
        optimizer.zero_grad() # clear gradients
        outputs = net(inputs) # forward step
        loss = criterion(outputs, labels)
        loss.backward() # backprop
        optimizer.step() # optimize weights

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        running_loss += loss.item()
        training_error = 1-correct_train/total_train
    
    
        if i % log_interval == log_interval-1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f, training error: %.1f%%' % 
                  (epoch + 1, i + 1, running_loss / log_interval, training_error*100))
            training_loss_log[int((epoch*len(train_labels)/log_interval)+(i+1)/log_interval-1)] = running_loss/log_interval
            training_error_log[int((epoch*len(train_labels)/log_interval)+(i+1)/log_interval-1)] = training_error
            running_loss = 0.0    
    
    training_acc = correct_train / total_train * 100
    training_acc_vect[epoch] = training_acc
    
    for i, data in enumerate(validloader, 0):
        inputs, labels = data # list of [inputs, labels]   
        
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()        
        
        outputs = net(inputs) # forward step        

        _, predicted = torch.max(outputs.data, 1)
        total_valid += labels.size(0)    
        correct_valid += (predicted == labels).sum().item()        

        # print statistics
        duration = time.time() - start_time        
    
    valid_acc = correct_valid / total_valid * 100
    valid_acc_vect[epoch] = valid_acc    
    
    print('Accuracy  of the network on the 50000 training images after epoch %d: train: %.2f valid: %.2f %% (%.1f sec)' % (
        epoch + 1, training_acc, valid_acc, duration))
    
    # your code here to calculate the validation error after each epoch
    
print('Finished Training')

Accuracy  of the network on the 50000 training images after epoch 1: train: 39.45 valid: 48.56 % (5.2 sec)
Accuracy  of the network on the 50000 training images after epoch 2: train: 54.45 valid: 55.57 % (10.3 sec)
Accuracy  of the network on the 50000 training images after epoch 3: train: 61.19 valid: 59.71 % (15.3 sec)
Accuracy  of the network on the 50000 training images after epoch 4: train: 66.07 valid: 59.86 % (20.3 sec)
Accuracy  of the network on the 50000 training images after epoch 5: train: 69.94 valid: 62.09 % (25.4 sec)
Accuracy  of the network on the 50000 training images after epoch 6: train: 73.14 valid: 62.99 % (30.4 sec)
Accuracy  of the network on the 50000 training images after epoch 7: train: 76.66 valid: 63.16 % (35.4 sec)
Accuracy  of the network on the 50000 training images after epoch 8: train: 79.74 valid: 63.15 % (40.5 sec)
Accuracy  of the network on the 50000 training images after epoch 9: train: 82.80 valid: 62.82 % (45.5 sec)
Accuracy  of the network on t

KeyboardInterrupt: 

In [ ]:
epoch_vect = np.linspace(1, num_epochs, num_epochs)

plt.figure(1)
plt.plot(epoch_vect, 100-training_acc_vect)
plt.plot(epoch_vect, 100-valid_acc_vect)
plt.grid('on')
plt.xlabel('epoch')
plt.ylabel('% error')
plt.title('DNN error (train & valid)')
plt.legend(['train','valid'])

print("Final Accuracy: train: %g valid %g" % (training_acc, valid_acc))

# Your code here to plot validation error

## Test the network

In [ ]:
MODELPATH = './cifar10_sample.pth.tar'
torch.save(net.state_dict(), MODELPATH)

In [ ]:
sum(p.numel() for p in net.parameters() if p.requires_grad)

## Estimate assignment grade on validation set

In [ ]:
network_complexity = {}
network_complexity['energy'] = 0.12881117 # Estimated energy in mJ.
network_complexity['latency'] = 0.03135800 # Number of cycles in Million (1e6).
network_complexity['activation'] = 49920 # Activation size in byte. 

In [ ]:
accuracy = 1-valid_error_vect[-1]
def get_score(accuracy, network_complexity):
    error_rate = (1-accuracy) * 100
#    error_rate = 20
    print(str(error_rate))
    loss = error_rate / 12 + network_complexity['latency'] / 0.3
    
    if error_rate > 50:
        return (0, loss)
    elif network_complexity['activation'] > 1000000:
        return (0, loss)
    else:
        score = 0
        if network_complexity['energy'] > 2:
            score += 0
        elif network_complexity['energy'] > 1.5:
            score += 5
        elif network_complexity['energy'] > 1:
            score += 10
        elif network_complexity['energy'] > 0.5:
            score += 15
        else:
            score += 20
            
        if network_complexity['latency'] > 1:
            score += 0
        elif network_complexity['latency'] > 0.5:
            score += 5
        elif network_complexity['latency'] > 0.25:
            score += 10
        elif network_complexity['latency'] > 0.1:
            score += 15
        else:
            score += 20
            
        if error_rate < 5:
            score += 80
        elif error_rate < 10:
            score += 70
        elif error_rate < 20:
            score += 60
        elif error_rate < 30:
            score += 50
        elif error_rate < 40:
            score += 40
        else:
            score += 30
            
        return (score, loss)

base_score, loss = get_score(accuracy, network_complexity)
    
# Calculated at the end of the competition based on everyones loss
competition_bonus = 0
final_score = base_score + (competition_bonus if base_score > 0 else 0)

print("Base Score: %d, Loss: %g" % (base_score, loss))
print("Final Score: %d" % (final_score))

In [ ]:
import datetime

def create_submission(labels):
    now = time.time()
    now_str = datetime.datetime.fromtimestamp(now).strftime('%m%d-%H%M%S')
    complexity_str = '{:.4f}-{:.4f}-{}'.format(network_complexity['energy'], \
                                               network_complexity['latency'], \
                                               network_complexity['activation']).replace('.', 'p')
    filename = 'submission-%s-%s.csv' % (complexity_str, now_str)
    with open(filename, 'w') as f:
        f.write("Id,Prediction\n")
        for n in range(labels.shape[0]):
            f.write("%d,%d\n" % (n,labels[n]))
    return now_str, filename

test_images = np.load("dataset/test_data.npy")

# convert to torch format
test_images = torch.from_numpy(np.expand_dims(test_images, axis=1))
test_result = torch.zeros(test_images.shape[0])

# test on validation data
with torch.no_grad():
    for i in range(test_images.shape[0]):
        images = test_images[i]
        images = (images/255.0)*2.0-1 # normalization [-1 to 1]
        images = images.float() # convert to torch format
        if use_cuda:
            images =images.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        test_result[i] = predicted
        if i % 10000 == 9999:
            print('Test data processed: %d/300000' % (i+1))

now_str, filename = create_submission(test_result.numpy())

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time, os
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

In [ ]:
trainset

In [ ]:
len(train_val_images)

In [ ]:
from torch.utils.data import Dataset

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def read_labels(label_file):
    labels = np.zeros(50000).astype(np.uint8)
    with open(label_file, 'r') as f:
        n, header_seen = 0, False
        for line in f:
            if not header_seen:
                header_seen = True
                continue
            labels[n] = int(line.strip().split(',')[1])
            n += 1
    return labels

train_labels = read_labels("dataset/train_labels.csv")

train_images = np.load("dataset/train_data.npy")

num_train = len(train_images)
indices = list(range(num_train))
split = 10000

# shuffle data
np.random.seed(6825)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

transform = torchvision.transforms.Compose(
    [torchvision.transforms.Normalize((0.5,), (0.5,))])

class CIFAR10Dataset(Dataset):
    def __init__(self, lbls, imgs, transform=None):
        self.transform=transform
        self.samples = [(torch.from_numpy(imgs[idx])*2.0/255.0 - 1,lbls[idx]) for idx in range(len(train_val_labels))]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


trainset = CIFAR10Dataset(train_labels,train_images,transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, sampler=train_sampler, shuffle=False)

validloader = torch.utils.data.DataLoader(trainset, batch_size=1, sampler=valid_sampler, shuffle=False)

In [ ]:
print(len(valid_idx))